# Cài đặt thư viện

In [22]:
import pandas as pd
import numpy as np
from numpy import array,asarray,zeros
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mhai0905/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mhai0905/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Đọc dữ liệu

In [23]:
df = pd.read_csv("IMDB_Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Ví dụ về tiền xử lý văn bản cho một review

#### Trước khi dự đoán mô hình ta tiến hành tiền xử lí dữ liệu. Để quá trình làm sạch dữ liệu diễn ra hiệu quả và quy mô với khối lượng dữ liệu lớn, em sẽ thử làm sạch dữ liệu ngẫu nhiên 1 câu trong tập dữ liệu.

In [24]:
example_review = df['review'].loc[23]
example_review

'First of all, let\'s get a few things straight here: a) I AM an anime fan- always has been as a matter of fact (I used to watch Speed Racer all the time in Preschool). b) I DO like several B-Movies because they\'re hilarious. c) I like the Godzilla movies- a lot.<br /><br />Moving on, when the movie first comes on, it seems like it\'s going to be your usual B-movie, down to the crappy FX, but all a sudden- BOOM! the anime comes on! This is when the movie goes WWWAAAAAYYYYY downhill.<br /><br />The animation is VERY bad & cheap, even worse than what I remember from SPEED RACER, for crissakes! In fact, it\'s so cheap, one of the few scenes from the movie I "vividly" remember is when a bunch of kids run out of a school... & it\'s the same kids over & over again! The FX are terrible, too; the dinosaurs look worse than Godzilla. In addition, the transition to live action to animation is unorganized, the dialogue & voices(especially the English dub that I viewed) was horrid & I was begging 

### 1. Xóa content HTML

In [25]:
soup = BeautifulSoup(example_review, "html.parser")
example_review = soup.get_text()
example_review

'First of all, let\'s get a few things straight here: a) I AM an anime fan- always has been as a matter of fact (I used to watch Speed Racer all the time in Preschool). b) I DO like several B-Movies because they\'re hilarious. c) I like the Godzilla movies- a lot.Moving on, when the movie first comes on, it seems like it\'s going to be your usual B-movie, down to the crappy FX, but all a sudden- BOOM! the anime comes on! This is when the movie goes WWWAAAAAYYYYY downhill.The animation is VERY bad & cheap, even worse than what I remember from SPEED RACER, for crissakes! In fact, it\'s so cheap, one of the few scenes from the movie I "vividly" remember is when a bunch of kids run out of a school... & it\'s the same kids over & over again! The FX are terrible, too; the dinosaurs look worse than Godzilla. In addition, the transition to live action to animation is unorganized, the dialogue & voices(especially the English dub that I viewed) was horrid & I was begging my dad to take the tape 

### 2. Sử dụng Biểu thức chính quy để xóa mọi thứ

In [26]:
example_review = re.sub('\[[^]]*\]', ' ',example_review)
example_review = re.sub('[^a-zA-Z]', ' ',example_review)
example_review

'First of all  let s get a few things straight here  a  I AM an anime fan  always has been as a matter of fact  I used to watch Speed Racer all the time in Preschool   b  I DO like several B Movies because they re hilarious  c  I like the Godzilla movies  a lot Moving on  when the movie first comes on  it seems like it s going to be your usual B movie  down to the crappy FX  but all a sudden  BOOM  the anime comes on  This is when the movie goes WWWAAAAAYYYYY downhill The animation is VERY bad   cheap  even worse than what I remember from SPEED RACER  for crissakes  In fact  it s so cheap  one of the few scenes from the movie I  vividly  remember is when a bunch of kids run out of a school      it s the same kids over   over again  The FX are terrible  too  the dinosaurs look worse than Godzilla  In addition  the transition to live action to animation is unorganized  the dialogue   voices especially the English dub that I viewed  was horrid   I was begging my dad to take the tape out o

### 3. Chuyển qua từ thường(LowerCase)

In [27]:
example_review = example_review.lower()
example_review

'first of all  let s get a few things straight here  a  i am an anime fan  always has been as a matter of fact  i used to watch speed racer all the time in preschool   b  i do like several b movies because they re hilarious  c  i like the godzilla movies  a lot moving on  when the movie first comes on  it seems like it s going to be your usual b movie  down to the crappy fx  but all a sudden  boom  the anime comes on  this is when the movie goes wwwaaaaayyyyy downhill the animation is very bad   cheap  even worse than what i remember from speed racer  for crissakes  in fact  it s so cheap  one of the few scenes from the movie i  vividly  remember is when a bunch of kids run out of a school      it s the same kids over   over again  the fx are terrible  too  the dinosaurs look worse than godzilla  in addition  the transition to live action to animation is unorganized  the dialogue   voices especially the english dub that i viewed  was horrid   i was begging my dad to take the tape out o

### 4. Tách từ (split)

In [28]:
example_review = example_review.split()
example_review

['first',
 'of',
 'all',
 'let',
 's',
 'get',
 'a',
 'few',
 'things',
 'straight',
 'here',
 'a',
 'i',
 'am',
 'an',
 'anime',
 'fan',
 'always',
 'has',
 'been',
 'as',
 'a',
 'matter',
 'of',
 'fact',
 'i',
 'used',
 'to',
 'watch',
 'speed',
 'racer',
 'all',
 'the',
 'time',
 'in',
 'preschool',
 'b',
 'i',
 'do',
 'like',
 'several',
 'b',
 'movies',
 'because',
 'they',
 're',
 'hilarious',
 'c',
 'i',
 'like',
 'the',
 'godzilla',
 'movies',
 'a',
 'lot',
 'moving',
 'on',
 'when',
 'the',
 'movie',
 'first',
 'comes',
 'on',
 'it',
 'seems',
 'like',
 'it',
 's',
 'going',
 'to',
 'be',
 'your',
 'usual',
 'b',
 'movie',
 'down',
 'to',
 'the',
 'crappy',
 'fx',
 'but',
 'all',
 'a',
 'sudden',
 'boom',
 'the',
 'anime',
 'comes',
 'on',
 'this',
 'is',
 'when',
 'the',
 'movie',
 'goes',
 'wwwaaaaayyyyy',
 'downhill',
 'the',
 'animation',
 'is',
 'very',
 'bad',
 'cheap',
 'even',
 'worse',
 'than',
 'what',
 'i',
 'remember',
 'from',
 'speed',
 'racer',
 'for',
 'crissak

### 5. Xử lí từ dừng(stopword)

In [29]:
example_review = [word for word in example_review if not word in set(stopwords.words('english'))]
example_review

['first',
 'let',
 'get',
 'things',
 'straight',
 'anime',
 'fan',
 'always',
 'matter',
 'fact',
 'used',
 'watch',
 'speed',
 'racer',
 'time',
 'preschool',
 'b',
 'like',
 'several',
 'b',
 'movies',
 'hilarious',
 'c',
 'like',
 'godzilla',
 'movies',
 'lot',
 'moving',
 'movie',
 'first',
 'comes',
 'seems',
 'like',
 'going',
 'usual',
 'b',
 'movie',
 'crappy',
 'fx',
 'sudden',
 'boom',
 'anime',
 'comes',
 'movie',
 'goes',
 'wwwaaaaayyyyy',
 'downhill',
 'animation',
 'bad',
 'cheap',
 'even',
 'worse',
 'remember',
 'speed',
 'racer',
 'crissakes',
 'fact',
 'cheap',
 'one',
 'scenes',
 'movie',
 'vividly',
 'remember',
 'bunch',
 'kids',
 'run',
 'school',
 'kids',
 'fx',
 'terrible',
 'dinosaurs',
 'look',
 'worse',
 'godzilla',
 'addition',
 'transition',
 'live',
 'action',
 'animation',
 'unorganized',
 'dialogue',
 'voices',
 'especially',
 'english',
 'dub',
 'viewed',
 'horrid',
 'begging',
 'dad',
 'take',
 'tape',
 'dvd',
 'vhs',
 'player',
 'thing',
 'kept',
 's

### 6. Nối lại các từ(join)

In [30]:
example_review = ' '.join(example_review)
example_review

'first let get things straight anime fan always matter fact used watch speed racer time preschool b like several b movies hilarious c like godzilla movies lot moving movie first comes seems like going usual b movie crappy fx sudden boom anime comes movie goes wwwaaaaayyyyy downhill animation bad cheap even worse remember speed racer crissakes fact cheap one scenes movie vividly remember bunch kids run school kids fx terrible dinosaurs look worse godzilla addition transition live action animation unorganized dialogue voices especially english dub viewed horrid begging dad take tape dvd vhs player thing kept surviving cracking jokes comments like robots joel mike mst k pick season honestly way barely enjoy movie survive time heck planning show another fellow otaku pal mine halloween b movie night stupid pretty painful watch unintentionally hilarious time giving movie improvement originally going give according grading scale means pretty much boring bad fun counting three unless find way 

### 7. Vector hóa văn bản: TfidfVectorizer

In [31]:
imdb = []
imdb.append(example_review)

In [32]:
tfidf_vecto = TfidfVectorizer()
review_tfidf_vecto = tfidf_vecto.fit_transform(imdb)
review_tfidf_vecto.toarray()

array([[0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.12379689,
        0.12379689, 0.06189845, 0.12379689, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.12379689,
        0.12379689, 0.06189845, 0.12379689, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.06189845,
        0.06189845, 0.12379689, 0.06189845, 0.06189845, 0.06189845,
        0.12379689, 0.18569534, 0.12379689, 0.06189845, 0.06189845,
        0.06189845, 0.12379689, 0.06189845, 0.12379689, 0.06189845,
        0.06189845, 0.06189845, 0.12379689, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.12379689,
        0.06189845, 0.24759378, 0.06189845, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06189845, 0.06189845,
        0.43328912, 0.12379689, 0.06189845, 0.06189845, 0.06189845,
        0.06189845, 0.06189845, 0.06189845, 0.06

 ### Vectơ hóa bằng kỹ thuật TF-IDF - trước khi tiền xử lý

In [36]:
tfidf_vecto = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,3))
tfidf_cv = tfidf_vecto.fit_transform(df['review'])
print('TfIdf_cv:',tfidf_cv.shape)

TfIdf_cv: (50000, 1689777)


# Tiền xử lý cho toàn bộ tập dữ liệu 50K

In [37]:
df['review'] = df['review'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
df['review'] = df['review'].apply(lambda x: re.sub('\[[^]]*\]', ' ', x))
df['review'] = df['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply(lambda x: x.split())
df['review'] = df['review'].apply(lambda x: [word for word in x if word not in set(stopwords.words('english'))])
df['review'] = df['review'].apply(lambda x: ' '.join(x))

/Users/mhai0905/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


# Chia tập dữ liệu

In [39]:
x_train, x_test, y_train, y_test= train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=2)

### Vectơ hóa bằng kỹ thuật TF-IDF - sau khi tiền xử lý

In [41]:
# Vectơ hóa bằng kỹ thuật TF-IDF - sau khi tiền xử lý
tfidf_vecto = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,3))
tfidf_vecto_train = tfidf_vecto.fit_transform(x_train)
tfidf_vecto_test = tfidf_vecto.transform(x_test)
print('TfIdf_cv_train:',tfidf_vecto_train.shape)
print('TfIdf_cv_test:',tfidf_vecto_test.shape)

TfIdf_cv_train: (35000, 600833)
TfIdf_cv_test: (15000, 600833)


# Huấn luyện mô hình

In [42]:
LSVC = LinearSVC()
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [43]:
print("Classification report of Linear SVC: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix of Linear SVC: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: " +str('{:04.2f}'.format(dochinhxac *100)) + "%")

Classification report of Linear SVC: 
               precision    recall  f1-score   support

    Positive       0.92      0.90      0.91      7499
    Negative       0.90      0.92      0.91      7501

    accuracy                           0.91     15000
   macro avg       0.91      0.91      0.91     15000
weighted avg       0.91      0.91      0.91     15000

Confusion Matrix of Linear SVC: 
 [[6751  748]
 [ 567 6934]]
Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: 91.23%


## Sau khi sử dụng mô hình LinearSVC để huấn luyện, ta được độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC là : 91.23%

# Build application: Predict sentiment

In [65]:
from sklearn.pipeline import Pipeline
clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=0.2, random_state=42))])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [66]:
clf.predict(['Wow, this is amazing lesson'])

array(['positive'], dtype=object)

In [67]:
import pickle
pickle.dump(clf, open('sentiment_analysis_mh.pkl', 'wb'))